# What is Trump yelling about?

## Setup

In [1]:
import twitter

In [2]:
# Define your api_key/secret and access_token_key/secret here

api = twitter.Api(consumer_key=api_key,
                      consumer_secret=api_secret,
                      access_token_key=access_token_key,
                      access_token_secret=access_token_secret)

In [3]:
api.VerifyCredentials()

User(ID=1961548171, ScreenName=kmarwahaha)

### Scrape last ~3200 tweets

In [4]:
def get_statuses(prev_statuses = None, screen_name = "realDonaldTrump", count=200):
    max_id = prev_statuses[-1].id-1 if prev_statuses else None
    return api.GetUserTimeline(screen_name=screen_name, max_id=max_id, count=count)

In [5]:
n = 16
temp = get_statuses()
all_statuses = temp
for _ in range(n):
    temp = get_statuses(temp)
    all_statuses += temp

In [6]:
# Earliest tweet scraped
all_statuses[-1]

Status(ID=888478670502932480, ScreenName=realDonaldTrump, Created=Fri Jul 21 19:19:56 +0000 2017, Text='Manufacturers’ record-high optimism reported in the 1st qtr has carried into the 2nd qtr of 2017 via @ShopFloorNAM:… https://t.co/G6QSAqjUeY')

### Parse date

In [7]:
#https://stackoverflow.com/questions/3682748/converting-unix-timestamp-string-to-readable-date-in-python
import datetime
def readable_date(unix_time):
    return datetime.datetime.fromtimestamp(int(unix_time)).strftime('%Y.%m.%d')

In [8]:
text_and_date = [(status.text, readable_date(status.created_at_in_seconds)) for status in all_statuses]

In [9]:
# Most recent tweet
text_and_date[0]

('Congratulations to Josh Hawley on your big Senate Primary win in Missouri. I look forward to working with you towar… https://t.co/gGWY572ROS',
 '2018.08.07')

## Analyze

In [10]:
def trim_results(text_and_date, filterx):
    return list(filter(
        lambda y: len(y[1]) > 0,
        [(date, 
            " ".join(filter(filterx, 
                            map(lambda x: x.replace('-', ' ').replace(u'\u2026', '').strip(), 
                                text.strip().split())
            ))
        ) for (text, date) in text_and_date]
    ))

### Approach 1: Find words that are uppercase, capitalized, hashtag'd, or end in "!"

In [11]:
filter1 = lambda x: x != u'&amp;' and (len(x) > 0 and x[0].upper() == x[0] 
                   or x.upper() == x or u'#' == x[0] or u'!' == x[-1])

In [12]:
trimmed1 = trim_results(text_and_date, filter1)

In [13]:
trimmed1[:20]

[('2018.08.07', 'Congratulations Josh Hawley Senate Primary Missouri. I'),
 ('2018.08.07',
  'Congratulations Bill Schuette. You Big November Governor Great St'),
 ('2018.08.07',
  'Congratulations STAR Republican Party, Senator John James. A'),
 ('2018.08.07', '.....Congratulations Troy Balderson Ohio. A race!'),
 ('2018.08.07', 'When I Ohio Troy Balderson, 64 36. That After'),
 ('2018.08.07', 'Today, 236th Purple Heart, Armed Forces'),
 ('2018.08.07',
  'RT @EricTrump: Ohio today! We Troy Balderson Congress. Visit'),
 ('2018.08.07', 'Ohio, Troy Balderson Congress. His Nancy Pelosi, Crime, B'),
 ('2018.08.07', 'The Iran These November'),
 ('2018.08.06', 'California'),
 ('2018.08.06', 'Democrats Open Borders ICE, Country'),
 ('2018.08.06', 'RT @realDonaldTrump: Presidential Approval  Better'),
 ('2018.08.06', 'John James Republican Star Senate Michigan. If'),
 ('2018.08.06', 'Governor Jerry Brown Free Flow North'),
 ('2018.08.06', 'Kris Kobach, Governor Great State Kansas. He'),
 ('201

Hmm, this isn't that insightful. Let's see what he's really yelling about.

### Approach 2: Just look for YELLING

In [14]:
#https://stackoverflow.com/questions/1265665/python-check-if-a-string-represents-an-int-without-using-try-except
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False
def IsASCII(u):
    try: 
        u.encode()
        return True
    except UnicodeEncodeError:
        return False

In [15]:
filter2 = lambda x: x.upper() == x and not RepresentsInt(x) and u'-' != x and x != u'RT' and len(x) > 3 and IsASCII(x)

In [16]:
trimmed2 = trim_results(text_and_date, filter2)

In [17]:
trimmed2[:20]

[('2018.08.07', 'STAR'),
 ('2018.08.06', 'ICE,'),
 ('2018.08.06', '100%'),
 ('2018.08.05', 'U.S.,'),
 ('2018.08.05', 'TRUE.'),
 ('2018.08.04', 'P.R.'),
 ('2018.08.04', 'U.S.,'),
 ('2018.08.04', 'HAPPY BIRTHDAY @USCG!'),
 ('2018.08.03', '@WWE'),
 ('2018.08.03', '@DRUDGE_REPORT:'),
 ('2018.08.03', '@DRUDGE_REPORT: +37,000...'),
 ('2018.08.03', '@DRUDGE_REPORT: RECORD 155,965,000 EMPLOYED'),
 ('2018.08.03', '@DRUDGE_REPORT:'),
 ('2018.08.03', '219,000 185,000 ADP”'),
 ('2018.08.03', '✅”US'),
 ('2018.08.03', '.@POTUS'),
 ('2018.08.03', '“THE RUSSIA HOAX, ILLICIT SCHEME CLEAR HILLARY CLINTON'),
 ('2018.08.03', 'MAKE AMERICA GREAT AGAIN!'),
 ('2018.08.03', '500,000'),
 ('2018.08.03', 'NASA,')]

That's a bit better.

### What's his yelling vocabulary like?

In [18]:
from collections import Counter

In [19]:
trump_yelling = Counter([x[1] for x in trimmed2])

In [20]:
[i for i in trump_yelling.most_common() if i[1] > 2]

[('U.S.', 76),
 ('#MAGA', 21),
 ('GREAT', 19),
 ('DACA', 19),
 ('MAKE AMERICA GREAT AGAIN!', 13),
 ('@FLOTUS', 12),
 ('NATO', 11),
 ('MAKING AMERICA GREAT AGAIN!', 10),
 ('MS 13', 10),
 ('DACA.', 10),
 ('@POTUS', 9),
 ('A.G.', 7),
 ('🇺🇸🇺🇸🇺🇸', 7),
 ('U.S.,', 6),
 ('.@POTUS', 6),
 ('FISA', 6),
 ('2018,', 6),
 ('ISIS', 6),
 ('FEMA', 6),
 ('#UNGA', 6),
 ('ICE,', 5),
 ('FBI.', 5),
 ('CUTS', 5),
 ('@FLOTUS:', 5),
 ('THANK', 5),
 ('D.C.', 5),
 ('2017,', 5),
 ('NEVER', 5),
 ('100%', 4),
 ('VERY', 4),
 ('JOBS, JOBS, JOBS!', 4),
 ('OPEC', 4),
 ('U.S.A.', 4),
 ('CNN.', 4),
 ('@FEMA', 4),
 ('ZERO', 4),
 ('T.V.', 4),
 ('HAPPY BIRTHDAY', 4),
 ('15 0', 4),
 ('#USA🇺🇸', 4),
 ('#SCOTUS', 3),
 ('AMERICA OPEN BUSINESS!', 3),
 ('VOTE', 3),
 ('WITCH HUNT!', 3),
 ('HEROES', 3),
 ('D.C.,', 3),
 ('LIVE #MAGA', 3),
 ('.@FLOTUS', 3),
 ('50%,', 3),
 ('ISIS,', 3),
 ('BORDER WALL', 3),
 ('#WEF18', 3),
 ('V.A.', 3)]

Makes sense. His whole YELLING vocab is below.

In [21]:
sorted(trump_yelling.keys())

['"20,000📈21,000📈22,000📈 #MAGA',
 '"46%',
 '"@POTUS',
 '"DEPLORABLES."',
 '"NO"',
 '#AMERICA FIRST',
 '#AMERICA FIRST!',
 '#APEC',
 '#APEC2017',
 '#APEC2017 @FLOTUS',
 '#ASEAN50 FAIR TRADE DEALS,',
 '#AZ08.',
 '#BYE BYE',
 '#CHANGETHELAWS',
 '#FAKE NEWS!',
 '#FEMA',
 '#GES2017',
 '#GES2017.',
 '#HELSINKI2018',
 '#ISIS',
 '#LESM',
 '#MAGA',
 '#MAGA @CPAC',
 '#MAGA WINNING,',
 '#MAGA!',
 '#MAGA🇺🇸',
 '#MS13 #MS13',
 '#NATO2018!',
 '#NATOS',
 '#S2155,',
 '#SCOTUS',
 '#SOTU',
 '#UNGA',
 '#USA',
 '#USAF B 1B',
 '#USA🇺🇸',
 '#USMC242',
 '#USVI.',
 '#USVI. @FEMA',
 '#WEF18',
 '$1.4',
 '$1.6',
 '$1.6 $700',
 '$1.6B U.S.A.',
 '$1.7 CASH',
 '$16.3+ 9.2%',
 '$20,000,000',
 '$500 DOWN,',
 '$700 $716',
 '$700,000',
 '$729M',
 '$800',
 '$972,000 GPS. $12,400,000 DNC.',
 "'16. C.M.",
 "'18.",
 '(#HR873.)',
 '($700,000)',
 '(110',
 '(5 10',
 '(@FLOTUS.)',
 '(AAA).',
 '(D VA),',
 '(D.C.) 4:00 P.M.',
 '(DACA)',
 '(DMD),',
 '(H.R. 2142)',
 '(HMX 1)',
 '(SEAL)',
 '(U.S.',
 '(USA)',
 '*NOT*',
 '.....',
 '...

### Saving and cleanup

In [22]:
%store all_statuses > all_statuses.txt
%store text_and_date > text_and_date.txt
%store trimmed1 > trimmed1.txt
%store trimmed2 > trimmed2.txt
%store trump_yelling > trump_yelling.txt

Writing 'all_statuses' (list) to file 'all_statuses.txt'.
Writing 'text_and_date' (list) to file 'text_and_date.txt'.
Writing 'trimmed1' (list) to file 'trimmed1.txt'.
Writing 'trimmed2' (list) to file 'trimmed2.txt'.
Writing 'trump_yelling' (Counter) to file 'trump_yelling.txt'.


In [23]:
%store -r